**F21BC**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
df=pd.read_csv('concrete_data.csv')


**Viewing the first few instances of the data**

In [ ]:
df.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


**Viewing the dimensionality of the dataset**

In [ ]:
df.shape

(1030, 9)

In [ ]:
df.dropna(inplace = True)
df.isnull().sum()

,0
cement,0
blast_furnace_slag,0
fly_ash,0
water,0
superplasticizer,0
coarse_aggregate,0
fine_aggregate,0
age,0
concrete_compressive_strength,0


In [ ]:
print(f'Duplicate rows: {df.duplicated().sum()}')

Duplicate rows: 25


In [ ]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)

**Scaling and Encoding**

In [ ]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)


In [ ]:
scaler = StandardScaler()

#intialise training and test sets
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print("First 5 rows of scaled training data:")
print(X_train_scaled[:5])

First 5 rows of scaled training data:
[[-0.7036142   0.7362629  -0.8810702   0.1799759  -1.0226942   1.32341753
  -0.17765612 -0.62455318]
 [-0.83112259 -0.84582676  1.04788182 -0.72981642  0.63596651  1.35948653
   0.32102131  0.88513966]
 [ 0.43140366  1.06620495 -0.8810702   0.38479825 -0.18498677 -1.33280675
   0.00745899 -0.28365479]
 [-1.05233032  0.66864226  1.10026551 -0.30588178  0.28412939  0.42298068
  -0.33758549 -0.51092038]
 [-1.17404287 -0.84582676  1.31904441  0.5419875   0.50193332 -1.24005789
   1.18741037 -0.28365479]]


**Artificial Neural Network (ANN)**
Below are the implementations of the ANN to train and predict concrete compressive strength

In [55]:
class activationFunction:
    def logisticFunction(x):
        return 1 / (1 + np.exp(-x))

    def reLuFunction(x):
        return np.maximum(0, x)

    def hyperbolicFunction(x):
        return np.tanh(x)

    def leakyReLU(x, alpha=0.01):
        return np.maximum(x, alpha * x)

    def elu(x, alpha=1.0):
        return np.where(x > 0, x, alpha * (np.exp(x) - 1))

class ArtificialNeuralNetwork:
    def __init__(self, layerSize, activationFunction):
        self.layerSize = layerSize
        self.activationFunction = activationFunction
        self.weights = [np.random.randn(layerSize[i], layerSize[i + 1]) for i in range(len(layerSize) - 1)]
        self.biases = [np.random.randn(1, layerSize[i + 1]) for i in range(len(layerSize) - 1)]

    def forwardPropagation(self, x):
        output = x
        for i in range(len(self.weights)):
            matrix_total = np.dot(output, self.weights[i]) + self.biases[i]
            output = self.activationFunction[i](matrix_total)  # Apply the ith activation function
        return output






*Layer size is structure to be 8 input layers, 16 neurons in first hidden layer, 8 for the second hidden layer, and 1 for the output layer.*
*This is to check the predicted values generated after forward propagation from the ANN*

In [56]:
layer_sizes = [8, 16, 8, 1]

activation_functions = [
    activationFunction.logisticFunction,
    activationFunction.reLuFunction,
    activationFunction.hyperbolicFunction,
]

ann = ArtificialNeuralNetwork(layer_sizes, activation_functions)

x_example = np.random.rand(8, 8)  # 8samples, 8 features
output = ann.forwardPropagation(x_example)
print("ANN Output (Sample Predictions):", output)


ANN Output (Sample Predictions): [[-0.99999982]
 [-0.99999626]
 [-0.9999906 ]
 [-0.99998553]
 [-0.99999984]
 [-0.99999717]
 [-0.99999998]
 [-0.99999992]]


**Loss Function**
*Since the problem domian is regression, MSE is utilised as the loss function*

In [ ]:
class lossFunction:
    def evaluate(self, y_pred, y_train):
        pass
class MeanSquaredError(lossFunction):
    def evaluate(self, y_pred, y_train):
        l=np.mean((y_pred - y_train) ** 2)
        return l

In [ ]:
loss_function = MeanSquaredError()

#output of forward propagation
y_pred = ann.forwardPropagation(X_train_scaled)

#use the output to calculate the loss function
print("Loss:", loss_function.evaluate(y_pred, y_train))

Loss: 1447.58422660566


**Implement PSO Algorithm**

In [57]:
class Particle:
    def __init__(self,vectorSize):
            self.particlePosition=np.random.rand(vectorSize)
            self.particleVelocity=np.random.rand(vectorSize)
            self.bestPosition=np.copy(self.particlePosition)
            self.informants=[]


In [58]:
def particleToAnn(particle, annLayers, activationFunctions):
    neuralNetwork = ArtificialNeuralNetwork(layerSize=annLayers, activationFunction=activationFunctions)
    weightBiasIndexCount = 0
    for i in range(len(annLayers) - 1):
        prevValue = annLayers[i]
        nextValue = annLayers[i + 1]
        weightRange = prevValue * nextValue
        weight = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + weightRange].reshape((prevValue, nextValue))
        weightBiasIndexCount += weightRange
        biases = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + nextValue].reshape((1, nextValue))
        weightBiasIndexCount += nextValue
        activation = activationFunctions[i]
        neuralNetwork.weights[i] = weight
        neuralNetwork.biases[i] = biases
    return neuralNetwork


In [ ]:
# Example of assessFitness
'''def assessFitness(particle, X, y, loss_function,predictions):
    # Implement forward propagation for the ANN represented by this particle
    # Use particle's position as ANN weights/biases
    # Calculate the loss (or error) using the provided loss_function
    # Return the computed fitness
    predictions = particle.forward_prop(X)  # Suppose each particle has a forward_prop method
    fitness = loss_function(predictions, y)
    return fitness'''


def assessFitness(particle,dataset,annLayers,activationFunctions,lossFunction):
    x, y = dataset
    ann=particleToAnn(particle,annLayers,activationFunctions)
    predictions = ann.forward(x.T)
    predicted_classes = (predictions > 0.5).astype(int)
    accuracy = np.mean(predicted_classes == y.reshape(-1, 1))
    return accuracy


In [59]:
# Testing the particleToAnn function
annLayers = [8, 16, 1]
activationFunctions = [activationFunction.logisticFunction,activationFunction.reLuFunction]
particle = Particle(161)
neuralNetwork = particleToAnn(particle, annLayers, activationFunctions)
x_example = np.random.rand(8, 8)
output = neuralNetwork.forwardPropagation(x_example)
print(output)

[[8.11021818]
 [8.3762438 ]
 [7.99499544]
 [7.93535654]
 [7.8823998 ]
 [7.8805294 ]
 [8.20397121]
 [7.92686223]]


In [43]:
import random
import numpy as np
class ParticleSwarmOptimisation:
    def __init__(self,swarmSize,alpha,beta,delta,omega,jumpSize,informantCount,vectorSize):
        self.swarmSize = swarmSize
        self.alpha = alpha
        self.beta = beta
        self.delta = delta
        self.omega = omega
        self.jumpSize = jumpSize
        self.informantCount = informantCount
        self.vectorSize = vectorSize
        self.global_best = None
        self.global_best_fitness = float('inf')


        # assign informants
        def initInformants(informantCount,particleArray):
            informants=[]
            for p in particleArray:
                potentialInformants=[]
                for potInf in particleArray:
                    if potInf!=p:
                        potentialInformants.append(potInf)
                for i in range(informantCount):
                     informants.append(random.choice(potentialInformants))
                p.informants=informants

        def get_best_informant(particle,dataset,annLayers,activationFunctions,lossFunction):
            bestInf=None
            bestFitnessInf=float('-inf')
            for i in particle.informants:
                fitness = assessFitness(i,dataset,annLayers,activationFunctions,lossFunction)
                if fitness > bestFitnessInf:
                    bestFitnessInf=fitness
                    bestInf=i
            bestInf.particlePosition=bestFitnessInf
            return bestInf.particlePosition



        #textbook code
        def psoOptimisation(swarmSize,alpha,beta,gamma,jumpSize,informantCount,vectorSize,dataSet):
            # stores all of the particles
            particleArray=[]
            for i in range(swarmSize):
                particleArray.append(Particle(vectorSize))
            best=None
            #initialising informants for the particles
            initInformants(informantCount,particleArray)
            while(True): #will change to do while loop
                # compare fitness
                for p in particleArray:
                    particleFitness=assessFitness(p)
                    bestFitness=assessFitness(best)
                    if best is None or particleFitness<bestFitness:
                        best=p
                for p in particleArray:
                    previousBest=p.bestPosition
                    informantsBest=get_best_informant
                    allBest=best.bestPosition
                    b = np.random.uniform(0.0, beta)
                    c = np.random.uniform(0.0, gamma)
                    d = np.random.uniform(0.0, delta)
                    updatedVelocity = alpha * p.velocity + b * (previousBest - p.position) + \
                            c * (informantsBest-p.position) + d * \
                    (allBest - p.position)

                    p.velocity=updatedVelocity
                    p.position+= jumpSize*updatedVelocity

[[6.8006315 ]
 [7.03973188]
 [6.82378219]
 [6.40968235]
 [6.54164057]
 [6.88042196]
 [6.77966653]
 [6.83015266]]
